# Import necessari:

In [1]:
from flickrapi import FlickrAPI
import json
from PIL import Image
from io import StringIO
import urllib
from torch.utils.data.dataset import Dataset
from PIL import Image
from os import path
import numpy as np
from pprint import pprint
from torch.utils.data import DataLoader
import os
import random
from torchvision import transforms
from skimage.transform import rescale, resize
from skimage import io
import torch
from torchvision import models
import torch.nn as nn
from torch.optim import SGD
from torchnet.meter import AverageValueMeter
from torchnet.logger import VisdomPlotLogger, VisdomSaver
from torch.autograd import Variable
from torch.nn import functional as F
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import confusion_matrix

# Dichiarazione costanti.

In [2]:
width = 256
height = 256
#Media e deviazione standard precalcolata per Squeezenet.
mean_squeezenet = [0.485, 0.456, 0.406]
std_squeezenet = [0.229, 0.224, 0.225]

# Dichiarazione di classi e funzioni.

In [3]:
class ScenesDataset(Dataset):
    def __init__(self,base_path,txt_list,transform=None):
        #conserviamo il path alla cartella contenente le immagini
        self.base_path=base_path
        #carichiamo la lista dei file
        #sarà una matrice con n righe (numero di immagini) e 2 colonne (path, etichetta)
        self.images = np.loadtxt(txt_list,dtype=str,delimiter=',')
        #conserviamo il riferimento alla trasformazione da applicare
        self.transform = transform
    def __getitem__(self, index):
        #recuperiamo il path dell'immagine di indice index e la relativa etichetta
        f,c = self.images[index]
        #carichiamo l'immagine utilizzando PIL
        im = Image.open(path.join(self.base_path, f)).convert('RGB')
        #Resize:
        im = im.resize((width,height))
        #se la trasfromazione è definita, applichiamola all'immagine
        if self.transform is not None:
            im = self.transform(im)
        #convertiamo l'etichetta in un intero
        label = int(c)
        #restituiamo un dizionario contenente immagine etichetta
        return {'image' : im, 'label':label}
    #restituisce il numero di campioni: la lunghezza della lista "images"
    def __len__(self):
        return len(self.images)

- Dato un oggetto di tipo ScenesDataset restituisce la media e la deviazione standard calcolata su tutte le immagini.

In [4]:
def get_mean_devst(dataset):
    m = np.zeros(3)
    for sample in dataset:
        m+= np.array(sample['image'].sum(1).sum(1)) #accumuliamo la somma dei pixel canale per canale
    #dividiamo per il numero di immagini moltiplicato per il numero di pixel
    m=m/(len(dataset)*width*height)
    #procedura simile per calcolare la deviazione standard
    s = np.zeros(3)
    for sample in dataset:
        s+= np.array(((sample['image']-torch.Tensor(m).view(3,1,1))**2).sum(1).sum(1))
    s=np.sqrt(s/(len(dataset)*width*height))
    #print("Medie",m)
    #print("Dev.Std.",s)
    return m, s

In [5]:
def IsImg(name):
    if str.__contains__(name, ".jpg"):
        return True
    return False

# Download immagini da flickr tramite Api search.
- Inserire FLICKR_PUBLIC e FLICKR_SECRET personali.

In [ ]:
FLICKR_PUBLIC = 'fdb1863f5162502cf08eafc41ddc43e5'
FLICKR_SECRET = '1bcbf94ddbf054d9'

flickr = FlickrAPI(FLICKR_PUBLIC, FLICKR_SECRET, format='parsed-json')
extras='url_m'

for pg in range(1,11):
    cats = flickr.photos.search(text='cat', page=pg, per_page=500, extras=extras, safe_search=1, content_type = 1, sort="interestingness-desc")
    photos_cats = cats['photos']
    if pg == 1:
        count = 0
    else:
        count = pg*500-501
    for i in cats['photos']['photo']:
        try:
            count += 1
            url = i['url_m']
            image = io.imread(url)
            #io.imshow(image)
            #io.show()
            io.imsave("cat/cat_" + str(count) + ".jpg", np.array(image))
            print("Cat number-->", count, "saved.")
        except:
            print("*******************")
            print("Errore...")
            print("*******************")
    
    dogs = flickr.photos.search(text='dog', page=pg, per_page=500, extras=extras, safe_search=1, content_type = 1, sort="interestingness-desc")
    photos_dogs = dogs['photos']
    if pg == 1:
        count = 0
    else:
        count = pg*500-501
    for i in dogs['photos']['photo']:
        try:
            count += 1
            url = i['url_m']
            image = io.imread(url)
            #io.imshow(image)
            #io.show()
            io.imsave("dog/dog_" + str(count) + ".jpg", np.array(image))
            print("Dog number-->", count, "saved.")
        except:
            print("*******************")
            print("Errore...")
            print("*******************")
print("Finished.")

# Creo i file train e test txt con le relative classi.

In [6]:
train = open("train.txt","w+")
test = open("test.txt","w+")
random.seed(1234)
entries = os.listdir('cat/')
stop = 0
for entry in entries:
    #if( stop == 200):
        #break
    stop += 1
    if IsImg(entry):
        if(random.randint(1, 100) <= 70):
            train.write("cat/" + entry + ", " + str(0) +"\n")
        else:
            test.write("cat/" + entry + ", " + str(0) + "\n")

entries = os.listdir('dog/')
stop = 0
for entry in entries:
    #if( stop == 200):
        #break
    stop += 1
    if IsImg(entry):        
        if(random.randint(1, 100) <= 70):
            train.write("dog/" + entry + ", " + str(1) +"\n")
        else:
            test.write("dog/" + entry + ", " + str(1) +"\n")

train.close()
test.close()

# Creo l'oggetto dataset

In [7]:
#Decommentare per ricalcolare se cambiano le immagini.
dataset_train = ScenesDataset('','train.txt',transform=transforms.ToTensor())
#mean_pre_trained, std_pre_trained = get_mean_devst(dataset_train)

#Database normalizzato per squeezenet preallenata
#print(mean_pre_trained)
#print(std_pre_trained)
mean_pre_trained = [0.47607468, 0.44641235, 0.39604222]
std_pre_trained = [0.27902635, 0.27250564, 0.27781429]

transformss = transforms.Compose([
                                      #transforms.Resize((width,height)),
                                      transforms.ToTensor(), 
                                      #transforms.Normalize(mean_pre_trained,std_pre_trained),
                                      transforms.Normalize(mean_squeezenet, std_squeezenet),
                                  ])
dataset_train_squeezenet = ScenesDataset('','train.txt',transform=transformss)
dataset_test_squeezenet = ScenesDataset('','test.txt',transform=transformss)

#Database normalizzato per regressione logistica.
transformss = transforms.Compose([
                                      #transforms.Resize((width,height)),
                                      transforms.ToTensor(), 
                                      #transforms.Normalize(mean_pre_trained,std_pre_trained),
                                      transforms.Normalize(mean_pre_trained, std_pre_trained),
                                  ])
dataset_train_logistic_regression = ScenesDataset('','train.txt',transform=transformss)
dataset_test_logistic_regression = ScenesDataset('','test.txt',transform=transformss)


'''
sample = dataset_train[0]
#l'immagine è 3 x 256 x 256 perché è una immagine a colori
print(sample)
print(sample['image'].shape)
print(sample['label'])
'''

"\nsample = dataset_train[0]\n#l'immagine è 3 x 256 x 256 perché è una immagine a colori\nprint(sample)\nprint(sample['image'].shape)\nprint(sample['label'])\n"

# Creo l'oggetto DataLoader

In [8]:
train_loader_squeezenet = DataLoader(dataset_train_squeezenet, batch_size=20, num_workers=0, shuffle=True)
test_loader_squeezenet = DataLoader(dataset_test_squeezenet, batch_size=20, num_workers=0)

train_loader_logistic_regression = DataLoader(dataset_train_logistic_regression, batch_size=20, num_workers=0, shuffle=True)
test_loader_logistic_regression = DataLoader(dataset_test_logistic_regression, batch_size=20, num_workers=0)

In [9]:
class LogisticRegression(torch.nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegression, self).__init__()
        self.linear = torch.nn.Linear(input_dim, output_dim)

    def forward(self, x):
        outputs = self.linear(x)
        return outputs
    
    
class LogisticRegressor(nn.Module):
    def __init__(self, in_features, ou_features):
        """Costruisce un regressore logistico.
        Input:
        in_features: numero di feature in input (es. 30)"""
        super(LogisticRegressor, self).__init__() #richiamo il costruttore della superclasse
        #questo passo è necessario per abilitare alcuni meccanismi automatici dei moduli di PyTorch
        self.linear = nn.Linear(in_features, ou_features) #il regressore logistico restituisce probabilità
        #quindi il numero di feature di output è "1"
        self.logistic = nn.Sigmoid()
    def forward(self,x):
        """Definisce come processare l'input x"""
        logits = self.linear(x)
        if self.training: #se il modulo è in fase di training
        #la proprietà "training" è messa a disposizione dai meccanismi
        #automatici dei moduli di PyTorch
            return logits
        else: #se siamo in fase di test, calcoliamo le probabilità
            return self.logistic(logits)

# Variabili di controllo per l'allenamento:

In [14]:
batch_size = 100
n_iters = 3000
epochs = n_iters / (len(dataset_train) / batch_size)
input_dim = width*height*3
output_dim = 2
lr_rate = 0.00001
channel = 3
momentum = 0.75

In [11]:
model_logistic_regression = LogisticRegression(input_dim, output_dim)
model_squeezenet = models.squeezenet1_0(pretrained=True)#(input_dim, output_dim)

In [12]:
def train_classification(model, train_loader, test_loader, lr=0.001, epochs=20, momentum=0.8, exp_name = 'experiment', logistic_regression = False):
    criterion = nn.CrossEntropyLoss()
    optimizer = SGD(model.parameters(),lr, momentum=momentum)
    loaders = {'train':train_loader, 'test':test_loader}
    losses = {'train':[], 'test':[]}
    accuracies = {'train':[], 'test':[]}
    
    
    loss_meter = AverageValueMeter()
    acc_meter = AverageValueMeter()
    loss_logger = VisdomPlotLogger('line', env=exp_name, opts={'title': 'Loss', 'legend':['train','test']})
    acc_logger = VisdomPlotLogger('line', env=exp_name, opts={'title': 'Accuracy','legend':['train','test']})
    visdom_saver = VisdomSaver(envs=[exp_name])

    if torch.cuda.is_available():
        model=model.cuda()
    
    for e in range(epochs):
        #print("Primo ciclo for.")
        for mode in ['train', 'test']:
            #print("Secondo ciclo for.")
            
            loss_meter.reset()
            acc_meter.reset()
            
            if mode=='train':
                model.train()
            else:
                model.eval()
            epoch_loss = 0
            epoch_acc = 0
            samples = 0
            #print("Mode-->",mode)
            for i, batch in enumerate(loaders[mode]):
                #trasformiamo i tensori in variabili
                if logistic_regression == True:
                    x=Variable(batch['image'].view(-1, width*height*channel), requires_grad=(mode=='train'))
                else:
                    x=Variable(batch['image'], requires_grad=(mode=='train'))

                y=Variable(batch['label'])
                if torch.cuda.is_available():
                    x, y = x.cuda(), y.cuda()
                    #print("Con cuda")
                #else:
                    #print("Senza cuda")
                output = model(x)
                #print(output)
                #print(y)
                l = criterion(output,y)
                if mode=='train':
                    l.backward()
                    optimizer.step()
                    optimizer.zero_grad()
                #print("L-->",l.item())
                acc = accuracy_score(y.cpu().data,output.cpu().max(1)[1].data)
                epoch_loss+=l.data.item()*x.shape[0]
                epoch_acc+=acc*x.shape[0]
                samples+=x.shape[0]
                '''print ("\r[%s] Epoch %d/%d. Iteration %d/%d. Loss: %0.2f. Accuracy: %0.2f\t\t\t\t\t" % \
                (mode, e+1, epochs, i, len(loaders[mode]), epoch_loss/samples, epoch_acc/samples),
                epoch_loss/samples,
                epoch_acc/samples,
                losses[mode].append(epoch_loss))'''
                accuracies[mode].append(epoch_acc)
                n = batch['image'].shape[0]
                loss_meter.add(l.item()*n,n)
                acc_meter.add(acc*n,n)
                #loss_logger.log(e+(i+1)/len(loaders[mode]), loss_meter.value()[0], name=mode)
                #acc_logger.log(e+(i+1)/len(loaders[mode]), acc_meter.value()[0], name=mode)


            loss_logger.log(e+1, loss_meter.value()[0], name=mode)
            acc_logger.log(e+1, acc_meter.value()[0], name=mode)
            if mode == "train":
                print(e, end = " ")
            #print("Fine secondo ciclo for")
        '''print("\r[%s] Epoch %d/%d. Iteration %d/%d. Loss: %0.2f. Accuracy: %0.2f\t\t\t\t\t" % \
        (mode, e+1, epochs, i, len(loaders[mode]), epoch_loss, epoch_acc))'''
        torch.save(model.state_dict(), "./" + exp_name + ".pth")
        print("Pesi aggiornati.")
    #print("Ho finito.")
    #restituiamo il modello e i vari log
    return model, (losses, accuracies)

In [ ]:
lenet_mnist, lenet_mnist_logs = train_classification(model_logistic_regression, 
                                                     epochs = 50, 
                                                     train_loader = train_loader_logistic_regression,
                                                     test_loader = test_loader_logistic_regression, 
                                                     logistic_regression = True,
                                                     exp_name = "Train_logistic_regression", 
                                                     lr = lr_rate, 
                                                     momentum = momentum)

In [15]:
result_squeezenet, result_logs_squeezenet = train_classification(model_squeezenet, 
                                                     epochs = 30, 
                                                     train_loader = train_loader_squeezenet,
                                                     test_loader = test_loader_squeezenet, 
                                                     logistic_regression = False,
                                                     exp_name = "Train_squeezenet_lr00001_momentum075", 
                                                     lr = lr_rate, 
                                                     momentum = momentum)

0 Pesi aggiornati.
1 Pesi aggiornati.


KeyboardInterrupt: 

In [ ]:
from sklearn.neighbors import KNeighborsClassifier as KNN
nn5 = KNN(5)
nn5.fit(X2_train,y2_train)
predicted_labels=nn5.predict(X2_test)
a = accuracy_score(y2_test,predicted_labels)
M = confusion_matrix(y2_test,predicted_labels)
print("5-NN, accuracy: %0.2f, Confusion Matrix:\n" %a)
print(M)

In [ ]:
# import the necessary packages
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from imutils import paths
import numpy as np
import argparse
import imutils
import cv2
import os

In [ ]:
def image_to_feature_vector(image, size=(32, 32)):
    # resize the image to a fixed size, then flatten the image into
    # a list of raw pixel intensities
    return cv2.resize(image, size).flatten()

In [ ]:
def extract_color_histogram(image, bins=(8, 8, 8)):
    # extract a 3D color histogram from the HSV color space using
    # the supplied number of `bins` per channel
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hist = cv2.calcHist([hsv], [0, 1, 2], None, bins,
    [0, 180, 0, 256, 0, 256])

    # handle normalizing the histogram if we are using OpenCV 2.4.X
    if imutils.is_cv2():
        hist = cv2.normalize(hist)

    # otherwise, perform "in place" normalization in OpenCV 3 (I
    # personally hate the way this is done
    else:
        cv2.normalize(hist, hist)

    # return the flattened histogram as the feature vector
    return hist.flatten()